# Trabalho Programação Linear 

## Leitura dos dados 

In [129]:
from pandas_ods_reader import read_ods

df = read_ods("variaveis_decisao.ods", 1)
df

,Item,Unidade,Local transporte,Variável de decisão,Volume [cm^3],Venda (R$),Compra,Lucro
0,Ovo de codorna,Dúzia,Kombi,x1,560.0,5.00,3.00,2.00
1,Ovo Branco,Dúzia,Kombi,x2,1458.0,5.00,3.50,1.50
2,Ovo Vermelho,Dúzia,Kombi,x3,1458.0,7.00,5.00,2.00
3,Coxa de Frango,Embalagem,Geladeira,x4,640.0,12.90,7.00,5.90
4,Picanha Suína,Embalagem,Geladeira,x5,630.0,26.90,18.80,8.10
5,Picanha Bovina,Embalagem,Geladeira,x6,1512.0,49.40,36.40,13.00
6,Miúdo de Porco,Embalagem,Geladeira,x7,13950.0,38.70,24.00,14.70
7,Pé de Frango,Caixa,Geladeira,x8,112404.0,85.00,34.00,51.00
8,Carré,Embalagem,Geladeira,x9,20160.0,81.12,48.00,33.12
9,Carne Moída,Embalagem,Geladeira,x10,15876.0,48.00,40.00,8.00


## Definição das variáveis 

In [33]:
volume = list(df['Volume [cm^3]'])
preco_venda = list(df['Venda (R$)'])
preco_compra = list(df['Compra'])
lucro = list(df['Lucro'])

## Definição das matrizes de Entrada 

### Função Objetiva

Iremos minimizar a -f(x1, ... , x22), onde f(x1, ... , x22) é a função que representa o valor total da venda.

In [40]:
Z = [x * -1 for x in preco_venda]

### Restrições

Nesta etapa iremos impor as restrições das variáves.

#### Restrição Geladeira

Nesta etapa selecionaremos os itens que irão dentro da geladeira, ou seja, de x4 a x22.

In [81]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item != 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
G = list(aux['Volume [cm^3]'])
max_G = 1369900

#### Restrição Kombi

Agora iremos impor as restrições para os itens que irão fora da geladeira. 

In [82]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item == 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
K = list(aux['Volume [cm^3]'])
max_K = 1984000

#### Fluxo de caixa

Existe um limite diário de mercadória de R$ 6000,00.

In [83]:
F = [x * -1 for x in preco_venda]
max_F = 6000

# Aplicando o Simplex 

In [84]:
import scipy
from scipy.optimize import linprog

In [112]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x1_bd = (0, None)
x_bounds = (x1_bd,)*len(c)

In [115]:
modelo_simplex = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='Simplex', options={"disp":True})

Optimization terminated successfully.
         Current function value: -76206.841270
         Iterations: 9


In [117]:
modelo_simplex

     con: array([], dtype=float64)
     fun: -76206.84126984127
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 0.00000000e+00, -2.32830644e-10,  8.22068413e+04])
  status: 0
 success: True
       x: array([3542.85714286,    0.        ,    0.        ,    0.        ,
       2174.44444444,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ])

In [124]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x1_bd = (144, 216)
x2_bd = (72, 108)
x3_bd = (48, 72)
x4_bd = (19.2, 28.8)
x5_bd = (0.8*144, 1.2*216)
x6_bd = (0.8*144, 1.2*216)
x7_bd = (0.8*144, 1.2*216)
x8_bd = (0.8*144, 1.2*216)
x9_bd = (0.8*144, 1.2*216)
x10_bd = (0.8*144, 1.2*216)
x11_bd = (0.8*144, 1.2*216)
x12_bd = (0.8*144, 1.2*216)
x13_bd = (0.8*144, 1.2*216)
x14_bd = (0.8*144, 1.2*216)
x15_bd = (0.8*144, 1.2*216)
x16_bd = (0.8*144, 1.2*216)
x17_bd = (0.8*144, 1.2*216)
x18_bd = (0.8*144, 1.2*216)
x19_bd = (0.8*144, 1.2*216)
x20_bd = (0.8*144, 1.2*216)
x21_bd = (0.8*144, 1.2*216)
x22_bd = (0.8*144, 1.2*216)
x_bounds = (x1_bd,
x2_bd,
x3_bd ,
x4_bd ,
x5_bd,
x6_bd,
x7_bd ,
x8_bd,
x9_bd,
x10_bd,
x11_bd,
x12_bd,
x13_bd,
x14_bd,
x15_bd,
x16_bd,
x17_bd,
x18_bd,
x19_bd,
x20_bd,
x21_bd,
x22_bd)

In [108]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, GLPK

In [109]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

In [111]:
x1 = LpVariable(name="x1", lowBound=0, cat="Integer")
x2 = LpVariable(name="x2", lowBound=0, cat="Integer")
x3 = LpVariable(name="x3", lowBound=0, cat="Integer")
x4 = LpVariable(name="x4", lowBound=0, cat="Integer")
x5 = LpVariable(name="x5", lowBound=0, cat="Integer")
x6 = LpVariable(name="x6", lowBound=0, cat="Integer")
x7 = LpVariable(name="x7", lowBound=0, cat="Integer")
x8 = LpVariable(name="x8", lowBound=0, cat="Integer")
x9 = LpVariable(name="x9", lowBound=0, cat="Integer")
x10 = LpVariable(name="x10", lowBound=0, cat="Integer")
x11 = LpVariable(name="x11", lowBound=0, cat="Integer")
x12 = LpVariable(name="x12", lowBound=0, cat="Integer")
x13 = LpVariable(name="x13", lowBound=0, cat="Integer")
x14 = LpVariable(name="x14", lowBound=0, cat="Integer")
x15 = LpVariable(name="x15", lowBound=0, cat="Integer")
x16 = LpVariable(name="x16", lowBound=0, cat="Integer")
x17 = LpVariable(name="x17", lowBound=0, cat="Integer")
x18 = LpVariable(name="x18", lowBound=0, cat="Integer")
x19 = LpVariable(name="x19", lowBound=0, cat="Integer")
x20 = LpVariable(name="x20", lowBound=0, cat="Integer")
x21 = LpVariable(name="x21", lowBound=0, cat="Integer")
x22 = LpVariable(name="x22", lowBound=0, cat="Integer")

In [ ]:
# Add the constraints to the model
model += (640*x4+630*x5+1512*x6+13950*x7+112404*x8+20160*x9+
          15876*x10+30618*x11+5985*x12+560*x13+4851*x14+ <= 20, "red_constraint")



In [ ]:
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")